In [ ]:
!python -m spacy download en_core_web_trf
!pip install spacy-transformers

2024-03-31 11:01:45.903789: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-31 11:01:46.020535: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-31 11:01:46.024825: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-03-31 11:01:46.024848: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if yo

In [ ]:
import spacy
from spacy.training import offsets_to_biluo_tags
import pandas as pd
from sklearn.metrics import classification_report

test_path = '/work/NER-test.tsv'
test_data = pd.read_csv(test_path, encoding='latin1', sep='\t')

nlp = spacy.load('en_core_web_trf')

/shared-libs/python3.9/py/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-03-31 12:03:25.861379: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-31 12:03:26.008542: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-31 12:03:26.012755: W tensorflow/stream_executor/platform/default/dso_loader.

In [ ]:
counter = 0
sentence = []
sentences = []
for index, row in test_data.iterrows():
    if row["sentence id"] == counter:
        sentence.append(row["token"])
    else:
        counter = row["sentence id"]
        sentences.append(" ".join(sentence))
        sentence = [row["token"]]
counter = row["sentence id"]
sentences.append(" ".join(sentence))
sentence = [row["token"]]
counter = 0
results = pd.DataFrame(columns=["token","predicted","test"])
for i,s in enumerate(sentences):
    doc = nlp(s)
    for i in range(len(doc)):
        if doc[i].ent_iob_ == "O":
            predicted = "O"
        else:
            predicted = doc[i].ent_iob_+ "-" +doc[i].ent_type_

        # Fix annotation error in test set
        if test_data.iloc[counter]["BIO NER tag"] == "B-PER":
            test = "B-PERSON"
        elif test_data.iloc[counter]["BIO NER tag"] == "I-PER":
            test = "I-PERSON" 
        else:
            test = test_data.iloc[counter]["BIO NER tag"]

        results.loc[counter] = [doc[i].text,predicted,test]
        counter += 1

print(results)

/shared-libs/python3.9/py/lib/python3.9/site-packages/torch/amp/autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
/shared-libs/python3.9/py/lib/python3.9/site-packages/torch/amp/autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
/shared-libs/python3.9/py/lib/python3.9/site-packages/torch/amp/autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
/shared-libs/python3.9/py/lib/python3.9/site-packages/torch/amp/autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User p

In [ ]:
classification_rep = classification_report(results["test"], results["predicted"])
print(classification_rep)

               precision    recall  f1-score   support

       B-DATE       1.00      1.00      1.00         1
       B-NORP       0.00      0.00      0.00         0
        B-ORG       1.00      1.00      1.00         3
     B-PERSON       1.00      1.00      1.00         6
B-WORK_OF_ART       1.00      0.75      0.86         4
       I-DATE       0.25      1.00      0.40         1
        I-ORG       1.00      1.00      1.00         6
     I-PERSON       1.00      1.00      1.00         3
I-WORK_OF_ART       1.00      0.78      0.88         9
            O       0.98      0.97      0.98       160

     accuracy                           0.96       193
    macro avg       0.82      0.85      0.81       193
 weighted avg       0.98      0.96      0.97       193

/shared-libs/python3.9/py/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` 

In [ ]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from nltk.corpus.reader import ConllCorpusReader

train_path = '/work/CONLL2003/'
train_data = ConllCorpusReader(train_path , 'train.txt', ['words', 'pos', 'ignore', 'chunk'])

test_path = '/work/NER-test.tsv'
test_data = pd.read_csv(test_path, encoding='latin1', sep='\t')

df_train = train_data
df_test = test_data
#print(len(df_train), len(df_test))
train_features = []
train_labels = []

for token, pos, ne_label in df_train.iob_words():
    feature_dict = {
        'word': token,
        'pos': pos
        
    }
    train_features.append(feature_dict)
    train_labels.append(ne_label)

test_features = []
test_labels = []

for index, row in df_test.iterrows():
    feature_dict = {
        'token': row['token']
        
    }
    test_features.append(feature_dict)
    test_labels.append(row['BIO NER tag'])

# Combine train and test features
combined_features = train_features + test_features

# Represent features using one-hot encoding
vectorizer = DictVectorizer(sparse=False)
one_hot_features = vectorizer.fit_transform(combined_features)

# Split the one-hot encoded features back into train and test
train_one_hot = one_hot_features[:len(train_features)]
test_one_hot = one_hot_features[len(train_features):]

#Naive Bayes

import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

vectorizer = DictVectorizer(sparse=False)
train_features_encoded = vectorizer.fit_transform(train_features)
test_features_encoded = vectorizer.transform(test_features)



# Model training
nb_classifier = MultinomialNB()
nb_classifier.fit(train_features_encoded, train_labels)

# Model evaluation
predicted_labels = nb_classifier.predict(test_features_encoded)
classification_rep = classification_report(test_labels, predicted_labels)

print(classification_rep)


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=7400df28-0aa0-427f-99a7-f8a604f99f5d' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>